# MODNet on the experimental and PBE datasets

In this model, we will see if training first the MODNet model on both the PBE and experimental datasets and then, fine tuning the model on the experimental dataset will improve the results.

In [1]:
from modnet.preprocessing import MODData
from modnet.models import MODNetModel
import numpy as np
import os
import copy
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau

In [2]:
from sklearn.model_selection import KFold
from modnet.preprocessing import MODData

def shuffle_MD(data,random_state=10):
    data = copy.deepcopy(data)
    ids = data.df_targets.sample(frac=1,random_state=random_state).index
    data.df_featurized = data.df_featurized.loc[ids]
    data.df_targets = data.df_targets.loc[ids]
    data.df_structure = data.df_structure.loc[ids]
    
    return data

def MDKsplit(data,n_splits=5,random_state=10):
    data = shuffle_MD(data,random_state=random_state)
    ids = np.array(data.structure_ids)
    kf = KFold(n_splits=n_splits,shuffle=True,random_state=random_state)
    folds = []
    for train_idx, val_idx in kf.split(ids):
        data_train = MODData(data.df_structure.iloc[train_idx]['structure'].values,data.df_targets.iloc[train_idx].values,target_names=data.df_targets.columns,structure_ids=ids[train_idx])
        data_train.df_featurized = data.df_featurized.iloc[train_idx]
        #data_train.optimal_features = data.optimal_features
        
        data_val = MODData(data.df_structure.iloc[val_idx]['structure'].values,data.df_targets.iloc[val_idx].values,target_names=data.df_targets.columns,structure_ids=ids[val_idx])
        data_val.df_featurized = data.df_featurized.iloc[val_idx]
        #data_val.optimal_features = data.optimal_features

        folds.append((data_train,data_val))
        
    return folds

def MD_append(md,lmd):
    md = copy.deepcopy(md)
    for m in lmd:
        md.df_structure.append(m.df_structure)
        md.df_targets.append(m.df_targets)
        md.df_featurized.append(m.df_featurized)
    return md

In [3]:
md_exp = MODData.load('exp_gap_all')
md_exp.df_targets.columns = ['gap']
md_pbe = MODData.load('pbe_gap.zip')
md_pbe.df_targets.columns = ['gap']


If you use the ChemEnv tool for your research, please consider citing the following reference(s) :
David Waroquiers, Xavier Gonze, Gian-Marco Rignanese, Cathrin Welker-Nieuwoudt, Frank Rosowski,
Michael Goebel, Stephan Schenk, Peter Degelmann, Rute Andre, Robert Glaum, and Geoffroy Hautier,
"Statistical analysis of coordination environments in oxides",
Chem. Mater., 2017, 29 (19), pp 8346-8360,
DOI: 10.1021/acs.chemmater.7b02766



INFO:root:Loaded <modnet.preprocessing.MODData object at 0x7f7126fe56a0> object, created with modnet version <=0.1.7
INFO:root:Loaded <modnet.preprocessing.MODData object at 0x7f7126fe5850> object, created with modnet version <=0.1.7


In [4]:
k = 5
random_state = 202010
folds = MDKsplit(md_exp,n_splits=k,random_state=random_state)
maes_combined = np.ones(5)
maes = np.ones(5)
for i,f in enumerate(folds):
    train = f[0]
    test = f[1]
    fpath = 'train_{}_{}'.format(random_state,i+1)
    if os.path.exists(fpath):
        train = MODData.load(fpath)
        train.df_targets.columns=['gap']
    else:
        train.feature_selection(n=-1)
        train.save(fpath)
        
    # assure no overlap
    assert len(set(train.df_targets.index).intersection(set(test.df_targets.index))) == 0
    
    md = MD_append(train,[md_pbe])
    
    model = MODNetModel([[['gap']]],{'gap':1})
    model.fit_preset(md,verbose=0)
    
    # first test on combined model
    pred = model.predict(test)
    true = test.df_targets
    error = pred-true
    error = error.drop(pred.index[((pred['gap']).abs()>20)]) # drop unrealistic values: happens extremely rarely
    mae = np.abs(error.values).mean()
    print('mae_combined')
    print(mae)
    maes_combined[i] = mae
    
    #fine tune on exp only
    rlr = ReduceLROnPlateau(monitor="loss", factor=0.5, patience=20, verbose=1, mode="auto", min_delta=0)
    es = EarlyStopping(monitor="loss", min_delta=0.001, patience=300, verbose=1, mode="auto", baseline=None,restore_best_weights=True)
    model.fit(train,lr=0.01, epochs = 1000, batch_size = 64, loss='mae', callbacks=[rlr,es], verbose=1)
    
    pred = model.predict(test)
    true = test.df_targets
    error = pred-true
    error = error.drop(pred.index[((pred['gap']).abs()>20)]) # drop unrealistic values: happens extremely rarely
    mae = np.abs(error.values).mean()
    print('mae')
    print(mae)
    maes[i] = mae
    

INFO:root:Loaded DeBreuck2020Featurizer featurizer.
INFO:root:Loaded DeBreuck2020Featurizer featurizer.
INFO:root:Loaded DeBreuck2020Featurizer featurizer.
INFO:root:Loaded DeBreuck2020Featurizer featurizer.
INFO:root:Loaded DeBreuck2020Featurizer featurizer.
INFO:root:Loaded DeBreuck2020Featurizer featurizer.
INFO:root:Loaded DeBreuck2020Featurizer featurizer.
INFO:root:Loaded DeBreuck2020Featurizer featurizer.
INFO:root:Loaded DeBreuck2020Featurizer featurizer.
INFO:root:Loaded DeBreuck2020Featurizer featurizer.
INFO:root:Loaded <modnet.preprocessing.MODData object at 0x7f71847b4160> object, created with modnet version 0.1.8~develop
INFO:root:Training preset #1/16
INFO:root:Compiling model...
INFO:root:Fitting model...
INFO:root:Validation loss: 0.357
INFO:root:Training preset #2/16
INFO:root:Compiling model...
INFO:root:Fitting model...
INFO:root:Validation loss: 0.341
INFO:root:Training preset #3/16
INFO:root:Compiling model...
INFO:root:Fitting model...
INFO:root:Validation loss: 

mae_combined
0.41596685527292826
Epoch 1/1000
34/34 [==============================] - 0s 2ms/step - loss: 0.1328 - mae: 0.1328
Epoch 2/1000
34/34 [==============================] - 0s 2ms/step - loss: 0.1081 - mae: 0.1081
Epoch 3/1000
34/34 [==============================] - 0s 2ms/step - loss: 0.1010 - mae: 0.1010
Epoch 4/1000
34/34 [==============================] - 0s 2ms/step - loss: 0.0920 - mae: 0.0920
Epoch 5/1000
34/34 [==============================] - 0s 2ms/step - loss: 0.1017 - mae: 0.1017
Epoch 6/1000
34/34 [==============================] - 0s 2ms/step - loss: 0.1503 - mae: 0.1503
Epoch 7/1000
34/34 [==============================] - 0s 2ms/step - loss: 0.1048 - mae: 0.1048
Epoch 8/1000
34/34 [==============================] - 0s 2ms/step - loss: 0.0939 - mae: 0.0939
Epoch 9/1000
34/34 [==============================] - 0s 2ms/step - loss: 0.0906 - mae: 0.0906
Epoch 10/1000
34/34 [==============================] - 0s 2ms/step - loss: 0.0871 - mae: 0.0871
Epoch 11/1000
34

INFO:root:Loaded <modnet.preprocessing.MODData object at 0x7f704bfafd30> object, created with modnet version 0.1.8~develop
INFO:root:Training preset #1/16
INFO:root:Compiling model...
INFO:root:Fitting model...
INFO:root:Validation loss: 0.441
INFO:root:Training preset #2/16
INFO:root:Compiling model...
INFO:root:Fitting model...
INFO:root:Validation loss: 0.386
INFO:root:Training preset #3/16
INFO:root:Compiling model...
INFO:root:Fitting model...
INFO:root:Validation loss: 0.328
INFO:root:Training preset #4/16
INFO:root:Compiling model...
INFO:root:Fitting model...
INFO:root:Validation loss: 0.362
INFO:root:Training preset #5/16
INFO:root:Compiling model...
INFO:root:Fitting model...
INFO:root:Validation loss: 0.385
INFO:root:Training preset #6/16
INFO:root:Compiling model...
INFO:root:Fitting model...
INFO:root:Validation loss: 0.352
INFO:root:Training preset #7/16
INFO:root:Compiling model...
INFO:root:Fitting model...
INFO:root:Validation loss: 0.316
INFO:root:Training preset #8/1

mae_combined
0.3440890430487301
Epoch 1/1000
34/34 [==============================] - 0s 2ms/step - loss: 0.1332 - mae: 0.1332
Epoch 2/1000
34/34 [==============================] - 0s 2ms/step - loss: 0.0977 - mae: 0.0977
Epoch 3/1000
34/34 [==============================] - 0s 2ms/step - loss: 0.0884 - mae: 0.0884
Epoch 4/1000
34/34 [==============================] - 0s 2ms/step - loss: 0.0940 - mae: 0.0940
Epoch 5/1000
34/34 [==============================] - 0s 2ms/step - loss: 0.0887 - mae: 0.0887
Epoch 6/1000
34/34 [==============================] - 0s 2ms/step - loss: 0.0878 - mae: 0.0878
Epoch 7/1000
34/34 [==============================] - 0s 2ms/step - loss: 0.1057 - mae: 0.1057
Epoch 8/1000
34/34 [==============================] - 0s 2ms/step - loss: 0.1148 - mae: 0.1148
Epoch 9/1000
34/34 [==============================] - 0s 2ms/step - loss: 0.1236 - mae: 0.1236
Epoch 10/1000
34/34 [==============================] - 0s 2ms/step - loss: 0.0958 - mae: 0.0958
Epoch 11/1000
34/

INFO:root:Loaded <modnet.preprocessing.MODData object at 0x7f70e9d76160> object, created with modnet version 0.1.8~develop
INFO:root:Training preset #1/16
INFO:root:Compiling model...
INFO:root:Fitting model...
INFO:root:Validation loss: 0.378
INFO:root:Training preset #2/16
INFO:root:Compiling model...
INFO:root:Fitting model...
INFO:root:Validation loss: 0.401
INFO:root:Training preset #3/16
INFO:root:Compiling model...
INFO:root:Fitting model...
INFO:root:Validation loss: 0.414
INFO:root:Training preset #4/16
INFO:root:Compiling model...
INFO:root:Fitting model...
INFO:root:Validation loss: 0.382
INFO:root:Training preset #5/16
INFO:root:Compiling model...
INFO:root:Fitting model...
INFO:root:Validation loss: 0.368
INFO:root:Training preset #6/16
INFO:root:Compiling model...
INFO:root:Fitting model...
INFO:root:Validation loss: 0.426
INFO:root:Training preset #7/16
INFO:root:Compiling model...
INFO:root:Fitting model...
INFO:root:Validation loss: 0.367
INFO:root:Training preset #8/1

mae_combined
0.4130800344786022
Epoch 1/1000
34/34 [==============================] - 0s 2ms/step - loss: 0.1708 - mae: 0.1708
Epoch 2/1000
34/34 [==============================] - 0s 2ms/step - loss: 0.1274 - mae: 0.1274
Epoch 3/1000
34/34 [==============================] - 0s 2ms/step - loss: 0.1385 - mae: 0.1385
Epoch 4/1000
34/34 [==============================] - 0s 2ms/step - loss: 0.1257 - mae: 0.1257
Epoch 5/1000
34/34 [==============================] - 0s 2ms/step - loss: 0.1353 - mae: 0.1353
Epoch 6/1000
34/34 [==============================] - 0s 2ms/step - loss: 0.1307 - mae: 0.1307
Epoch 7/1000
34/34 [==============================] - 0s 2ms/step - loss: 0.1504 - mae: 0.1504
Epoch 8/1000
34/34 [==============================] - 0s 2ms/step - loss: 0.1438 - mae: 0.1438
Epoch 9/1000
34/34 [==============================] - 0s 1ms/step - loss: 0.1590 - mae: 0.1590
Epoch 10/1000
34/34 [==============================] - 0s 2ms/step - loss: 0.1602 - mae: 0.1602
Epoch 11/1000
34/

INFO:root:Loaded <modnet.preprocessing.MODData object at 0x7f70ee81e520> object, created with modnet version 0.1.8~develop
INFO:root:Training preset #1/16
INFO:root:Compiling model...
INFO:root:Fitting model...
INFO:root:Validation loss: 0.384
INFO:root:Training preset #2/16
INFO:root:Compiling model...
INFO:root:Fitting model...
INFO:root:Validation loss: 0.374
INFO:root:Training preset #3/16
INFO:root:Compiling model...
INFO:root:Fitting model...
INFO:root:Validation loss: 0.691
INFO:root:Training preset #4/16
INFO:root:Compiling model...
INFO:root:Fitting model...
INFO:root:Validation loss: 0.385
INFO:root:Training preset #5/16
INFO:root:Compiling model...
INFO:root:Fitting model...
INFO:root:Validation loss: 0.412
INFO:root:Training preset #6/16
INFO:root:Compiling model...
INFO:root:Fitting model...
INFO:root:Validation loss: 0.415
INFO:root:Training preset #7/16
INFO:root:Compiling model...
INFO:root:Fitting model...
INFO:root:Validation loss: 0.377
INFO:root:Training preset #8/1

mae_combined
0.4123182648115069
Epoch 1/1000
34/34 [==============================] - 0s 1ms/step - loss: 0.1461 - mae: 0.1461
Epoch 2/1000
34/34 [==============================] - 0s 1ms/step - loss: 0.1274 - mae: 0.1274
Epoch 3/1000
34/34 [==============================] - 0s 1ms/step - loss: 0.1336 - mae: 0.1336
Epoch 4/1000
34/34 [==============================] - 0s 1ms/step - loss: 0.1357 - mae: 0.1357
Epoch 5/1000
34/34 [==============================] - 0s 1ms/step - loss: 0.1270 - mae: 0.1270
Epoch 6/1000
34/34 [==============================] - 0s 1ms/step - loss: 0.1140 - mae: 0.1140
Epoch 7/1000
34/34 [==============================] - 0s 1ms/step - loss: 0.1219 - mae: 0.1219
Epoch 8/1000
34/34 [==============================] - 0s 1ms/step - loss: 0.1400 - mae: 0.1400
Epoch 9/1000
34/34 [==============================] - 0s 1ms/step - loss: 0.1285 - mae: 0.1285
Epoch 10/1000
34/34 [==============================] - 0s 1ms/step - loss: 0.1494 - mae: 0.1494
Epoch 11/1000
34/

INFO:root:Loaded <modnet.preprocessing.MODData object at 0x7f706d254c40> object, created with modnet version 0.1.8~develop
INFO:root:Training preset #1/16
INFO:root:Compiling model...
INFO:root:Fitting model...
INFO:root:Validation loss: 0.456
INFO:root:Training preset #2/16
INFO:root:Compiling model...
INFO:root:Fitting model...
INFO:root:Validation loss: 0.431
INFO:root:Training preset #3/16
INFO:root:Compiling model...
INFO:root:Fitting model...
INFO:root:Validation loss: 0.436
INFO:root:Training preset #4/16
INFO:root:Compiling model...
INFO:root:Fitting model...
INFO:root:Validation loss: 0.418
INFO:root:Training preset #5/16
INFO:root:Compiling model...
INFO:root:Fitting model...
INFO:root:Validation loss: 0.413
INFO:root:Training preset #6/16
INFO:root:Compiling model...
INFO:root:Fitting model...
INFO:root:Validation loss: 0.433
INFO:root:Training preset #7/16
INFO:root:Compiling model...
INFO:root:Fitting model...
INFO:root:Validation loss: 0.395
INFO:root:Training preset #8/1

mae_combined
0.3692314854174752
Epoch 1/1000
34/34 [==============================] - 0s 2ms/step - loss: 0.1362 - mae: 0.1362
Epoch 2/1000
34/34 [==============================] - 0s 1ms/step - loss: 0.1065 - mae: 0.1065
Epoch 3/1000
34/34 [==============================] - 0s 2ms/step - loss: 0.0865 - mae: 0.0865
Epoch 4/1000
34/34 [==============================] - 0s 1ms/step - loss: 0.0870 - mae: 0.0870
Epoch 5/1000
34/34 [==============================] - 0s 2ms/step - loss: 0.0913 - mae: 0.0913
Epoch 6/1000
34/34 [==============================] - 0s 2ms/step - loss: 0.0903 - mae: 0.0903
Epoch 7/1000
34/34 [==============================] - 0s 2ms/step - loss: 0.0884 - mae: 0.0884
Epoch 8/1000
34/34 [==============================] - 0s 2ms/step - loss: 0.0950 - mae: 0.0950
Epoch 9/1000
34/34 [==============================] - 0s 2ms/step - loss: 0.1025 - mae: 0.1025
Epoch 10/1000
34/34 [==============================] - 0s 1ms/step - loss: 0.0979 - mae: 0.0979
Epoch 11/1000
34/

In [5]:
maes_combined.mean()

0.3909371366058485

In [6]:
maes.mean()

0.3859477956153032

#### Conclusion

No significant improvement.